# StaticFrame 0.8.14 to 0.8.30


## Keeping up with StaticFrame

* All major features and fixes are on "micro" releases (0.8.30 to 0.8.31)
* Backward-incompatible releases (0.8 to 0.9) tend to focus just on API changes
* Release Notes: https://github.com/InvestmentSystems/static-frame/releases
* Code: https://github.com/InvestmentSystems/static-frame/commits/master

# StaticFrame 0.9 Coming Soon

Backward incompatible changes
* Dissallow creating `Index` with `datetime64` arrays; require using `datetime64` `Index` types.
* Change `Bus.__init__()` to no longer accept a Series but take internally managed components.
* `Batch` no longer automatically promotes container types, must explicitly use `via_container`.
* `*.from_overlay()` will be renamed `*.from_overlay_na()` to permit adding a `*.from_overlay_falsy()`.

## Overview of New Features

### New features
* `*.rank_*` methods.
* Support for `falsy` in many places where `na`
* Lots of new methods to ``via_dt`` interfaces.
* String slicing with ``*.via_str[]``
* Added ``Yarn``
* Unpersisting on `Bus`, `Yarn`, `Quilt`.
* NPZ, NPY formats for `Frame` and `Bus`. 
* `Quilt.sample()`
* ``IndexHiearchy.relabel_at_depth()``

 
### Extension to existing features
* `sort_*` methods now take `ascending` as iterable of Booleans.
* `*.count()` methods now have `skipfalsy` and `unique` parameters.
* `*.equals()` methods distringuish between ``datetime64`` unit.
* Added `dtypes` parameter to `Frame.from_pandas()`
* `Frame.iter_group()` and `Frame.iter_group_items()` take a `drop` parameter.
* `Frame.unset_index` works with `IndexHierarchy`
* Improved `__repr__` for `Quilt`
* Added `index_constructors` and `columns_constructors` to many interfaces.
* Added `index_constructor` parameters to many interfaces. 

### Performance Enhancements
* All `Bus` iteration when ``max_persist`` is greater than one.
* `Bus` internal architecture enhancements.
* `Bus` used weakrefs to avoid re-loading `Frame` already in-memory.
* `Frame.iter_group()` and `Frame.iter_group_items()`
* `Frame.pivot()`
* `Frame.from_concat()`
* `Frame.to_pandas()` creates ``pd.RangeIndex`` for ``IndexAutoFactory``-created indices.

### Better Errors:
* Incorrectly formed ``Batch`` iterables raise ``BatchIterableInvalid``

### Linux (or WSL) only
* Support for `VisiData` via `Frame.to_visidata()`, `Bus.to_visidata()`

### Advanced Features
* `FrameGO.via_fill_value()` works in `__setitem__()`
* `IndexAutoFactory` takes a `size` parameter.
* `IndexDefaultFactory` permits specifying ``name`` of default `Index`


## Falsy v. Na

The family of `*na*` functions, which process `None`, `np.nan`, `np.nat` now have corresponding `*falsy*` functions, which process `None`, `np.nan`, `np.nat`, `0`, `False`, and `""`. 

> I know, `np.nan` is not Falsy

### On Frame, Series

* `*.isfalsy()`, 
* `*.notfalsy()`, 
* `*.dropfalsy()`, 
* `*.fillfalsy`.
* `*.fillfalsy_forward()`
* `*.fillfalsy_backward()`
* `*.fillfalsy_leading()`
* `*.fillfalsy_trailing()`



## Index Constructor Arguments.

Index constructor arguments are now available in (hopefully) all places where needed. 


Added `index_constructor`:

* `*.from_concat_items()`
* `apply`
* `apply_pool`
* `map_any`
* `map_fill`
* `map_all`


Note that interfaces that hae `*_depth` arguments use `*_constructors` arguments, not `_constructor` arguments.

* ``StoreConfig``
* ``Frame.from_sql()``
* ``Frame.from_structured_array()``
* ``Frame.from_delimited()``
* ``Frame.from_csv()``
* ``Frame.from_clipboard``
* ``Frame.from_tsv()``
* ``Frame.from_xlsx()``
* ``Frame.from_sqlite()``
* ``Frame.from_hdf5()``
* ``Frame.from_arrrow()``
* ``Frame.from_parquet()``

## Extension to `via_dt` interfaces

All faster than using `datetime` objects.

### New attributes:
* ``hour``
* ``minute``
* ``second``

### New methods:
* ``is_month_start()``
* ``is_month_end()``
* ``is_year_start()``
* ``is_year_end()``
* ``is_quarter_start()``
* ``is_quarter_end()`` 
* ``quarter()``

## NPY and NPZ

The `to_npz()` and `to_npy()` interfaces offer a `consolidate_blocks` parameter.

### Frame interfaces

* `Frame.to_npz()`
* `Frame.from_npz()`
* `Frame.to_npy()`
* `Frame.from_npy()`
* `Frame.from_npy_mmap()` (0.8.31)

### Bus interfaces
* ``Bus.to_zip_npz()``
* ``Bus.from_zip_npz()``
* ``Quilt.to_zip_npz()``
* ``Quilt.from_zip_npz()``
* ``Batch.to_zip_npz()``
* ``Batch.from_zip_npz()``
* ``Yarn.to_zip_npz()``



## Unpersisting

Force "forgetting" all loaded `Frame`, regardless of `max_persist` configuration.


### Interfaces
* `Bus.unpersist()`
* `Yarn.unpersist()`
* `Quilt.unpersist()`

